In [1]:
# 필요한 기본 패키지 준비
# 데이터처리를 위한 패키지
import numpy as np
import pandas as pd

# encoding 문제 해결을 위한 패키지
import chardet

# 날짜 처리를 위한 패키지
import datetime as dt

# 파일 위치 이름 가져오기 위한 패키지
import glob

In [2]:
# year=2016
# month=1
# day=1

def yyyy(year):
    year_str='{:0>4}'.format(year) # 년도 4개의 자리수로
    return year_str

def mm(month):
    mm='{:0>2}'.format(month) # 월 2개의 자리수로(자리수가 부족하면 숫자0으로 자리 메꾸기, 예: 1월 => 1 => 01
    return mm

def dd(day):
    dd='{:0>2}'.format(day)
    return dd


# 원본 파일 위치
# D:/project/contest/data/original/uv/
#                                      201606, 201607...의 규칙으로 저장됨
# glob을 위한 전용 함수 : 각 년도 폴더의 csv자료만 검색 리스트로 리턴
def filePath(year, month):
    path = 'E:/Python/original/uv/'
    file = path+yyyy(year)+'/'+yyyy(year)+mm(month)+'/*.csv'
    return file

# print(filePath(2016,6))
def find_encoding(fname):
    r_file = open(fname, 'rb').read()
    result = chardet.detect(r_file)
    charenc = result['encoding']
    return charenc


['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

In [64]:
# 11억 : 서울특별시
# 41억 : 경기도
# 28억 : 인천광역시
# '1100000000':'서울', '4100000000':'경기도', '2800000000':'인천'
# 16개의 컬럼(0~15)

# uv데이터 관측지점 정보 가져오기.
excel = 'E:/Python/original/air/OpenAPI_20190115/loc_tb_air_UV.xlsx'
areaCode = pd.read_excel(excel)

# 1번, 16번 의미 없음
# index로 보면 1~14번 컬럼만
areaCode = areaCode.iloc[:,1:15]

# '3단계' column에는 행정구역 단위가 '동'단위 이다.
# 즉, '시/군/구' 단위까지만 자료를 가져오겠다는 뜻.
areaCode = areaCode.loc[areaCode['3단계'].isna()]
# areaCode.head(4)

# 지점번호, 시/도, 시/군/구 column만
areaCode = areaCode.iloc[:,:3]

# 향후 테이블 merge를 위한 0번 column 이름 '행정구역코드' -> '지점번호'로 수정
areaCode.columns = ['지점번호','1단계', '2단계']
# dtype 통일
areaCode['지점번호'] = areaCode['지점번호'].astype('str')
# areaCode.head(5)

In [65]:
# 2016~2018/3~11월사이의 데이터 들을 가져온다
# 다행이도 glob에서 특정 path가 없으면 자체적으로 건너뛰고 다음으로 넘어가준다.
file_list = list()
for year in range(2016,2019):
    for month in range(3,12):
        file_list.extend(glob.glob(filePath(year, month)))

In [116]:
find_encoding(file_list[0])

str

## 데이터 전 처리 법 확인 

In [117]:
df = pd.read_csv('E:/Python/original/uv/2016/201606\\FCT_IDX_A07_2016061706.csv', encoding='euc-kr',sep='#',dtype='str', usecols=[1,2,3,4,5])
df['date'] = pd.to_datetime(df['날짜'].str[:-2])
df2 = pd.read_csv('E:/Python/original/uv/2016/201606\\FCT_IDX_A07_2016061718.csv', encoding='euc-kr',sep='#',dtype='str', usecols=[1,2,3,4,5])
df2['date'] = pd.to_datetime(df2['날짜'].str[:-2])

# df와 df2를 합치기 df 아래 df2를 붙이는 형식으로
df3 = pd.concat([df, df2], ignore_index=True)

# 지점명 추가
df_joined = pd.merge(df3, areaCode, on='지점번호',how='left')

# '1단계' : 특별시/광역시/도 가 존재하는 데이터만
df_joined = df_joined.loc[df_joined['1단계'].notnull()]

# 서울('11...'), 경기('41...'), 인천('28...') 지역 정보만 가져오기
# gs/lalavla데이터가 서울/경기/인천 지역만 제공 되기 때문.
df_joined = df_joined.loc[(df_joined['지점번호'].str.startswith('11')) #서울
                          | (df_joined['지점번호'].str.startswith('41')) # 경기
                          | (df_joined['지점번호'].str.startswith('28'))].reset_index(drop=True) #인천/원래index미사용
# 결과 확인
df_joined.tail(1)

,지점번호,날짜,오늘,내일,모레,date,1단계,2단계
155,4183000000,2016061718,NaN,8,6,2016-06-17,경기도,양평군


## 데이터 종합

In [118]:
# 모든 데이터가 결집될 df
uv_2016_2018 = pd.read_csv(file_list2016[0], encoding='euc-kr',sep='#',dtype='str', usecols=[1,2,3,4,5])
uv_2016_2018['date'] = pd.to_datetime(uv_2016_2018['날짜'].str[:-2])
uv_2016_2018 = uv_2016_2018.loc[(uv_2016_2018['지점번호'].str.startswith('11')) #서울
                          | (uv_2016_2018['지점번호'].str.startswith('41')) # 경기
                          | (uv_2016_2018['지점번호'].str.startswith('28'))].reset_index(drop=True)
# uv_2016_2018.head()

# 일단 2016년만
# file_list2016 = list()

# for month in range(3,12):
#     file_list2016.extend(glob.glob(filePath(2016, month)))

# file_list, 1번부터. 0번은 위에서 이미 읽음.
for index in range(1,len(file_list)):
    if index%100==0:
        print(index,'번째 파일 처리중...')
    # 2016자료는 전부 'euc-kr'
    my_encoding = find_encoding(file_list[index])
    df = pd.read_csv(file_list[index], encoding=my_encoding,sep='#',dtype='str', usecols=[1,2,3,4,5])
    df['date'] = pd.to_datetime(df['날짜'].str[:-2])
    df = df.loc[(df['지점번호'].str.startswith('11')) #서울
                          | (df['지점번호'].str.startswith('41')) # 경기
                          | (df['지점번호'].str.startswith('28'))].reset_index(drop=True)
    uv_2016_2018=pd.concat([uv_2016_2018, df], ignore_index=True) 
print(len(file_list),'개 파일 완료')  
# uv_2016_2018.tail(1)
# 2016년 데이터 종합 확인
# 대충 1430개의 파일

10 번째 파일 처리중...
20 번째 파일 처리중...
30 번째 파일 처리중...
40 번째 파일 처리중...
50 번째 파일 처리중...
60 번째 파일 처리중...
70 번째 파일 처리중...
80 번째 파일 처리중...
90 번째 파일 처리중...
100 번째 파일 처리중...
110 번째 파일 처리중...
120 번째 파일 처리중...
130 번째 파일 처리중...
140 번째 파일 처리중...
150 번째 파일 처리중...
160 번째 파일 처리중...
170 번째 파일 처리중...
180 번째 파일 처리중...
190 번째 파일 처리중...
200 번째 파일 처리중...
210 번째 파일 처리중...
220 번째 파일 처리중...
230 번째 파일 처리중...
240 번째 파일 처리중...
250 번째 파일 처리중...
260 번째 파일 처리중...
270 번째 파일 처리중...
280 번째 파일 처리중...
290 번째 파일 처리중...
300 번째 파일 처리중...
310 번째 파일 처리중...
320 번째 파일 처리중...
330 번째 파일 처리중...
340 번째 파일 처리중...
350 번째 파일 처리중...
360 번째 파일 처리중...
370 번째 파일 처리중...
380 번째 파일 처리중...
390 번째 파일 처리중...
400 번째 파일 처리중...
410 번째 파일 처리중...
420 번째 파일 처리중...
430 번째 파일 처리중...
440 번째 파일 처리중...
450 번째 파일 처리중...
460 번째 파일 처리중...
470 번째 파일 처리중...
480 번째 파일 처리중...
490 번째 파일 처리중...
500 번째 파일 처리중...
510 번째 파일 처리중...
520 번째 파일 처리중...
530 번째 파일 처리중...
540 번째 파일 처리중...
550 번째 파일 처리중...
560 번째 파일 처리중...
570 번째 파일 처리중...
580 번째 파일 처리중...
590 번째 파일 처리중...
600 번째

In [126]:
# 중간저장
uv_2016_2018.to_csv('E:/Python/original/uv/uv_2016_2018.csv', encoding='utf-8', index=False)

In [123]:
uv_2016_2018.isnull().sum()

지점번호         0
날짜           0
오늘      871643
내일           0
모레           0
date         0
dtype: int64

In [128]:
df_joined = pd.merge(uv_2016_2018, areaCode, on='지점번호',how='left')

# '1단계' : 특별시/광역시/도 가 존재하는 데이터만
df_joined = df_joined.loc[df_joined['1단계'].notnull()]

# 서울('11...'), 경기('41...'), 인천('28...') 지역 정보만 가져오기
# gs/lalavla데이터가 서울/경기/인천 지역만 제공 되기 때문.
df_joined = df_joined.loc[(df_joined['지점번호'].str.startswith('11')) #서울
                          | (df_joined['지점번호'].str.startswith('41')) # 경기
                          | (df_joined['지점번호'].str.startswith('28'))].reset_index(drop=True) #인천/원래index미사용
# 결과 확인
df_joined = df_joined.loc[(df_joined['지점번호'].str.startswith('11')) #서울
                          | (df_joined['지점번호'].str.startswith('41')) # 경기
                          | (df_joined['지점번호'].str.startswith('28'))].reset_index(drop=True) #인천/원래index미사용

In [129]:
df_joined.tail(10)

,지점번호,날짜,오늘,내일,모레,date,1단계,2단계
111296,4155000000,2018113018,NaN,2,1,2018-11-30,경기도,안성시
111297,4157000000,2018113018,NaN,2,1,2018-11-30,경기도,김포시
111298,4159000000,2018113018,NaN,2,1,2018-11-30,경기도,화성시
111299,4161000000,2018113018,NaN,2,1,2018-11-30,경기도,광주시
111300,4163000000,2018113018,NaN,2,1,2018-11-30,경기도,양주시
111301,4165000000,2018113018,NaN,2,1,2018-11-30,경기도,포천시
111302,4167000000,2018113018,NaN,2,1,2018-11-30,경기도,여주시
111303,4180000000,2018113018,NaN,2,1,2018-11-30,경기도,연천군
111304,4182000000,2018113018,NaN,2,1,2018-11-30,경기도,가평군
111305,4183000000,2018113018,NaN,2,1,2018-11-30,경기도,양평군


In [131]:
df_joined.isnull().sum()

지점번호        0
날짜          0
오늘      55692
내일          0
모레          0
date        0
1단계         0
2단계      4281
dtype: int64

In [132]:
# 지점명 추가
# 처리한 df저장
df_joined.to_csv('E:/Python/original/uv/uv_2016_2018.csv', encoding='utf-8', index=False)

# 결측치 처리를 해봅시다

In [3]:
# 필요한 기본 패키지 준비
# 데이터처리를 위한 패키지
import numpy as np
import pandas as pd

# encoding 문제 해결을 위하 ㄴ패키지
import chardet

# 날짜 처리를 위한 패키지
import datetime as dt

# 파일 위치 이름 가져오기 위한 패키지
import glob

In [49]:
uv = pd.read_csv('E:/Python/original/uv/uv_2016_2018.csv', encoding='utf-8',parse_dates=[5])
uv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111306 entries, 0 to 111305
Data columns (total 8 columns):
지점번호    111306 non-null int64
날짜      111306 non-null int64
오늘      55614 non-null float64
내일      111306 non-null int64
모레      111306 non-null int64
date    111306 non-null datetime64[ns]
1단계     111306 non-null object
2단계     107025 non-null object
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 6.8+ MB


In [51]:
uv.loc[uv['오늘'].isnull()].head()

,지점번호,날짜,오늘,내일,모레,date,1단계,2단계
78,1100000000,2016061718,NaN,8,6,2016-06-17,서울특별시,NaN
79,1111000000,2016061718,NaN,8,6,2016-06-17,서울특별시,종로구
80,1114000000,2016061718,NaN,8,6,2016-06-17,서울특별시,중구
81,1117000000,2016061718,NaN,8,6,2016-06-17,서울특별시,용산구
82,1120000000,2016061718,NaN,8,6,2016-06-17,서울특별시,성동구


In [52]:
print(list(uv['2단계'].unique()))
print(len(uv['2단계'].unique()))

[nan, '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '동구', '연수구', '남동구', '부평구', '계양구', '서구', '강화군', '옹진군', '수원시장안구', '수원시권선구', '수원시팔달구', '수원시영통구', '성남시수정구', '성남시중원구', '성남시분당구', '의정부시', '안양시만안구', '안양시동안구', '광명시', '평택시', '동두천시', '안산시상록구', '안산시단원구', '고양시덕양구', '고양시일산동구', '고양시일산서구', '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시', '의왕시', '하남시', '용인시처인구', '용인시기흥구', '용인시수지구', '파주시', '이천시', '안성시', '김포시', '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군', '양평군']
75


In [89]:
# 데이터 확인 결과 : 그 날의 자외선 지수는 오전에 처리. 오후에는 그날의 자외선 지수가 없음
# .mean()을 사용하면 결측치 제외하고 계산함.
# mean()으로 하루의 자외선지수 
# uv.iloc[:,[0,2,3,4,5,6,7]].loc[uv['date']=='2016-06-17']\
# .groupby(['date','지점번호','1단계','2단계']).mean().reset_index().head(2)
p_uv = uv.iloc[:,[0,2,3,4,5,6,7]].groupby(['date','지점번호','1단계','2단계']).mean().reset_index()
p_uv.tail(2)

# missing_day = list(p_uv.loc[p_uv['오늘'].isnull(),'date'].unique())
# array(['2016-08-03T00:00:00.000000000', '2018-08-25T00:00:00.000000000'], dtype='datetime64[ns]')
# 2016-08-03, 2018-08-25 없음
# 전날의 '내일' 예측 수치로 대체

# 75개소 관측지점 : 75행마다 날자가 바뀜
yesterday = p_uv['내일'].shift(periods=75)

# 데이터 정제전 각 컬럼의 길이 확인
# print(len(yesterday)==len(p_uv['내일'])) # True

p_uv['오늘_new'] = np.where(p_uv['오늘'].isnull(), yesterday, p_uv['오늘'])
####### 출처: https://rfriend.tistory.com/262 [R, Python 분석과 프로그래밍 (by R Friend)] #######
p_uv['오늘'] = p_uv['오늘_new']
p_uv.drop(columns='오늘_new',inplace=True)
p_uv.isnull().sum()

True


date    0
지점번호    0
1단계     0
2단계     0
오늘      0
내일      0
모레      0
dtype: int64

In [91]:
p_uv.to_csv('E:/Python/처리된 csv자료/p_uv.csv', encoding='utf-8', index=False)

In [78]:
?np.where

In [71]:
?pd.DataFrame.shift